In [1]:
import pandas as pd
import numpy as np

C:\Users\yanhui\AppData\Local\Temp\ipykernel_3464\2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# pip install gurobipy
import gurobipy as gp
from gurobipy import GRB

options = {
    "WLSACCESSID": "31598bc4-0b23-474f-86d4-af032d437137",
    "WLSSECRET": "bd7ad348-f8d6-452e-b615-ebc88d920e41",
    "LICENSEID": 2498404,
}

In [67]:
def dataPerDay(df,date):
    df = df[df['t0'] == date]
    C1 = df['C1']
    C2 = df['C2']
    K1 = df['K1']
    K2 = df['K2']
    S0 = df['Adj_S0'].unique()
    return S0[0], C1, C2, K1, K2, df
data = pd.read_csv('data_20180119_20180420.csv',index_col=0)
dateList = data['t0'].unique()
S0,C1,C2,K1,K2,df = dataPerDay(data,dateList[0])

### Grid for stock price
current setting S_{T2}, S_{T1} from range [1100,1300] and 20\*20 discretization

In [6]:
def gen_Grid(S_0,epsilon,n):
    S_min = S_0-epsilon
    S_max = S_0+epsilon
    S = np.linspace(S_min, S_max, num=n+1)
    return S

In [68]:
N_j = 20
epsi = 200
S = gen_Grid(S0,epsi,N_j)

### Optimization Variables
variables to optimize: lambda_1, lambda_2, d, \Delta_1, \Delta_0

### Model Construction
objective: min$d+\sum_{i=1}^{N_1}\lambda_{i,1}\Pi_{i,1}+\sum_{i=1}^{N_2}\lambda_{i,2}\Pi_{i,2}$

constrains: for each grid point (S_i,S_j), we have

$d+\sum_{n=1}^{N_1}\lambda_{n,1}(S_i-K_{n,1})^++\sum_{n=1}^{N_2}\lambda_{n,2}(S_j-K_{n,2})^++\Delta_{0}\cdot(S_i-S_0)+\Delta_{1}^i\cdot(S_j-S_i) >= \Phi(S_i,S_j)$

Current payoff $\Phi(S_i,S_j) = S_j - S_i$

In [69]:
def payoff(S1,S2):
    return 100
#     return S2-S1
def const_Model(df,S0,S,C1,C2,K1,K2):
    m = gp.Model("Option Pricing")
    lamb1 = m.addVars(df.index,name="lamb1")
    lamb2 = m.addVars(df.index,name="lamb2")
    d = m.addVars(['d'],name="d")
    delta0 = m.addVars(['delta0'],name="delta0")
    delta1 = m.addVars(np.array(S).flatten(),name="delta1")
    
    m.setObjective(d['d']+sum(lamb1[r]*C1[r]+lamb2[r]*C2[r] for r in df.index))
    m.ModelSense = GRB.MINIMIZE
    
    for S_i in S:
        for S_j in S:
            m.addConstr(d['d']+sum(lamb1[r]*max(S_i-K1[r],0)+lamb2[r]*max(S_j-K2[r],0) for r in df.index)+delta0['delta0']*(S_i-S0)+delta1[S_i]*(S_j-S_i) >= payoff(S_i,S_j),'C_{}{}'.format(S_i,S_j))
            m.update()
    m.optimize()
    return m.ObjVal

### Test Different Payoff
For 13 days of t0, test the optimization method in following 3 results:

Test results of following three payoff patterns: 1. constant payoff, 2. S2-S1, 3.S2

The expected price currently without interest rates should be 1.constant, 2. 0, 3. S0

In [78]:
## constant payoff 
def payoff(S1,S2):
    return 200
ObjVals = [] 
for i in range(len(dateList)):
    S0,C1,C2,K1,K2,df = dataPerDay(data,dateList[i])
    S = gen_Grid(S0,epsi,N_j)
    val = const_Model(df,S0,S,C1,C2,K1,K2)
    ObjVals.append(val)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows, 63 columns and 9681 nonzeros
Model fingerprint: 0xc710e98b
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [8e-01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 20 rows and 5 columns
Presolve time: 0.01s
Presolved: 421 rows, 58 columns, 9404 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.037500e+03   0.000000e+00      0s
       1    2.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.01 work units)
Optimal objective  2.000000000e+02
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800


Optimize a model with 441 rows, 63 columns and 8988 nonzeros
Model fingerprint: 0xd81a5090
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [7e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 2e+02]
Presolve removed 20 rows and 5 columns
Presolve time: 0.01s
Presolved: 421 rows, 58 columns, 8724 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   6.887500e+03   0.000000e+00      0s
       1    2.0000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.000000000e+02
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows, 63 columns and 8883 nonzeros
Model fingerprint: 0x452200e6
Coefficient statis

In [79]:
ObjVals

[200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0,
 200.0]

In [73]:
def payoff(S1,S2):
    return S2-S1
ObjVals = [] 
for i in range(len(dateList)):
    S0,C1,C2,K1,K2,df = dataPerDay(data,dateList[i])
    S = gen_Grid(S0,epsi,N_j)
    val = const_Model(df,S0,S,C1,C2,K1,K2)
    ObjVals.append(val)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows, 63 columns and 9681 nonzeros
Model fingerprint: 0x9ace71f1
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [8e-01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+02]
Presolve removed 51 rows and 5 columns
Presolve time: 0.01s
Presolved: 390 rows, 58 columns, 8424 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.257500e+03   0.000000e+00      0s
      19    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 19 iterations and 0.02 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 580


Optimize a model with 441 rows, 63 columns and 8988 nonzeros
Model fingerprint: 0x61767916
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [7e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 4e+02]
Presolve removed 51 rows and 5 columns
Presolve time: 0.01s
Presolved: 390 rows, 58 columns, 7767 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.813750e+03   0.000000e+00      0s
      19    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 19 iterations and 0.01 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows, 63 columns and 8883 nonzeros
Model fingerprint: 0x55e78c57
Coefficient stati

In [74]:
ObjVals

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [80]:
def payoff(S1,S2):
    return S2
ObjVals = [] 
for i in range(len(dateList)):
    S0,C1,C2,K1,K2,df = dataPerDay(data,dateList[i])
    S = gen_Grid(S0,epsi,N_j)
    val = const_Model(df,S0,S,C1,C2,K1,K2)
    ObjVals.append(val)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows, 63 columns and 9681 nonzeros
Model fingerprint: 0xcb181e5f
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [8e-01, 4e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 1e+03]
Presolve removed 20 rows and 5 columns
Presolve time: 0.01s
Presolved: 421 rows, 58 columns, 9404 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   2.928319e+04   0.000000e+00      0s
      29    1.1890100e+03   0.000000e+00   0.000000e+00      0s

Solved in 29 iterations and 0.01 seconds (0.01 work units)
Optimal objective  1.189010010e+03
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 580

Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows, 63 columns and 8988 nonzeros
Model fingerprint: 0x0267ae1d
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [7e-01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+03, 2e+03]
Presolve removed 20 rows and 5 columns
Presolve time: 0.01s
Presolved: 421 rows, 58 columns, 8724 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.441986e+04   0.000000e+00      0s
      28    1.3048599e+03   0.000000e+00   0.000000e+00      0s

Solved in 28 iterations and 0.01 seconds (0.01 work units)
Optimal objective  1.304859924e+03
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 441 rows

In [81]:
ObjVals

[1189.010009765625,
 1204.1999816894531,
 1209.5899963378906,
 1229.1400146484377,
 1246.8699645996094,
 1252.699966430664,
 1254.3299865722656,
 1275.897917209795,
 1305.2000427246096,
 1304.8599243164065,
 1295.0,
 1293.3200073242188,
 1286.6339211071156]

In [82]:
data['Adj_S0'].unique()

array([1189.01000977, 1204.19998169, 1209.58999634, 1229.14001465,
       1246.8699646 , 1252.69996643, 1254.32998657, 1276.67999268,
       1305.20004272, 1304.85992432, 1295.        , 1293.32000732,
       1294.57992554])